In [19]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 8.2 MB 10.8 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=fe4f0e8c1b26d4a8479c44ae32c0f324b8e359ae9d28bc0f11280884cb330ca0
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [17]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.5 MB/s 


In [69]:
import pandas as pd
import re
import numpy as np
import nltk
import pymorphy2
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from multiprocessing import Pool
from gensim import corpora
from gensim.models import *

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
df = pd.read_csv('/content/rospot.csv', names=['text'])

In [16]:
display(df.head(3), print(f'lenght: {df.shape[0]}'))

lenght: 246


,text
0,0
1,Между мной и «ПАО «Ростелеком» был заключен до...
2,"Здравствуйте, при заказе товаров в wildberries..."


None

In [22]:
morph = pymorphy2.MorphAnalyzer()

In [46]:
words_regex = re.compile('\w+')
sw = stopwords.words('russian')

class Preprocessing():
  def __init__(self, word_regex, stopwords, morph):
    self.words_regex = word_regex
    self.stopwords = stopwords
    self.morph = morph

  def find_words(self, text):
    tokens = self.words_regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 2]

  def lemmatize(self, words):
    lemmas = [self.morph.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in self.stopwords 
            and w.isalpha()]

  def preprocess(self, txt):
    return (self.lemmatize(self.find_words(txt)))

In [47]:
df.text.iloc[8]

'Добрый день. Был куплет тур в августе 2022, в сентябре объявили мобилизацию. Мы написали заявление на аннулирование тура, заявление принято тур оператором. В какой срок тур оператор должен вернуть денежные средства уплаченные за тур?'

In [50]:
preprocessing_txt = Preprocessing(words_regex, sw, morph)

In [51]:
print(preprocessing_txt.preprocess(df.text.iloc[1]))

['пао', 'ростелеком', 'заключить', 'договор', 'предоставление', 'услуга', 'интернет', 'ip', 'телевидение', 'пользование', 'тв', 'приставка', 'факт', 'заявка', 'виртуальный', 'договор', 'сайт', 'компания', 'получить', 'удаться', 'дать', 'договор', 'лицевой', 'счёт', 'исправно', 'выплачивать', 'плата', 'абонентский', 'стоимость', 'услуга', 'тариф', 'ежемесячно', 'выплата', 'тв', 'приставка', 'июнь', 'год', 'заявление', 'заявка', 'действие', 'договор', 'оказание', 'услуга', 'прекратить', 'причина', 'смена', 'место', 'жительство', 'установить', 'закон', 'порядок', 'прекращение', 'договор', 'просьба', 'менеджер', 'работа', 'клиент', 'досрочно', 'выкупить', 'тв', 'приставка', 'выплатить', 'остаться', 'стоимость', 'однако', 'июль', 'появиться', 'задолженность', 'услуга', 'интернет', 'сумма', 'руб', 'копейка', 'форма', 'электронный', 'письмо', 'просьба', 'оплатить', 'июль', 'обращение', 'служба', 'клиентский', 'поддержка', 'просьба', 'разъяснить', 'задолженность', 'менеджер', 'предоставить', '

In [52]:
preprocessed_text = list(tqdm(map(preprocessing_txt.preprocess, df['text']), total=len(df)))

  0%|          | 0/246 [00:00<?, ?it/s]

In [53]:
df['pre_text'] = preprocessed_text
df.sample(3)

,text,pre_text
201,При покупке машины была навязана дополнительно...,"[покупка, машина, навязать, дополнительно, усл..."
168,Пгт. Ревда во Победы д.27 Мурманской области....,"[пгт, ревда, победа, мурманский, область, цоко..."
237,"Здравствуйте, скажите пожалуйста, пластиковые ...","[здравствуйте, сказать, пожалуйста, пластиковы..."


In [54]:
#!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
# Модель использует векторное представление документов, например, мешок слов (bow), поэтому сперва создадим словарь:

In [58]:
dictionary = corpora.Dictionary(df['pre_text'])

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('lenta.dict')

In [62]:
# Векторизуем документы:

In [59]:
corpus = [dictionary.doc2bow(text) for text in df['pre_text']]
corpora.MmCorpus.serialize('lenta.model', corpus) 

In [63]:
# Обучаем модель

In [77]:
%time 
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=4, chunksize=50, update_every=1, passes=2)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 11 µs


/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [78]:
# Получившиеся темы

In [79]:
lda.show_topics(num_topics=10, num_words=10, formatted=True)

[(0,
  '0.031*"доставка" + 0.029*"квартира" + 0.026*"это" + 0.025*"магазин" + 0.025*"компания" + 0.024*"сторона" + 0.024*"ребёнок" + 0.023*"год" + 0.021*"здравствуйте" + 0.021*"мочь"'),
 (1,
  '0.049*"день" + 0.046*"возврат" + 0.038*"товар" + 0.033*"средство" + 0.030*"заказ" + 0.020*"право" + 0.020*"денежный" + 0.019*"добрый" + 0.017*"сайт" + 0.016*"который"'),
 (2,
  '0.046*"магазин" + 0.035*"товар" + 0.032*"это" + 0.030*"купить" + 0.029*"вернуть" + 0.027*"деньга" + 0.022*"возврат" + 0.020*"продавец" + 0.019*"сказать" + 0.018*"год"'),
 (3,
  '0.033*"просить" + 0.030*"это" + 0.020*"товар" + 0.019*"адрес" + 0.018*"проверка" + 0.017*"документ" + 0.017*"дом" + 0.017*"дать" + 0.017*"ул" + 0.015*"здравствуйте"')]

In [80]:
# Визуализация пространства тем

In [81]:
#!pip install pyLDAvis

In [82]:
%time 
space = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(space)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 11.4 µs
